In [1]:
from pgmpy.models import FactorGraph
from pgmpy.factors.discrete import DiscreteFactor
from pgmpy.inference.ExactInference import BeliefPropagation
import numpy as np

/home/takeshi/.pyenv/versions/3.9.4/envs/snn_bayes/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/takeshi/.pyenv/versions/3.9.4/envs/snn_bayes/lib/python3.9/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
import math
import sys
sys.path.append("../CSP/")
from sudoku import Sudoku

sudoku = Sudoku.make_sudoku()

In [3]:
sudoku_size = 9
root_sudoku_size = int(math.sqrt(sudoku_size))
pow_two_mask_except_zero = [True if (i & (i - 1)) == 0 and i != 0 else False for i in range(2**sudoku_size)] #True if 1 of K representation is valid. False in Otherwise
enforce_rate = 0.99
factor_prob = np.where(pow_two_mask_except_zero, enforce_rate, 1 - enforce_rate)
number_of_random_variable_state = [2 for _ in range(sudoku_size)]

In [5]:
graph = FactorGraph()
graph.add_nodes_from(["{}_{}_{}".format(y, x, num) for y in range(sudoku_size) for x in range(sudoku_size) for num in range(sudoku_size)])
# print("="*50)
#each cell constraint
for y in range(sudoku_size):
    for x in range(sudoku_size):
        target_node = ["{}_{}_{}".format(y, x, num) for num in range(sudoku_size)]
        # print(target_node)
        factor = DiscreteFactor(target_node, number_of_random_variable_state, factor_prob)
        graph.add_factors(factor)
        graph.add_edges_from([(target_node[i], factor) for i in range(sudoku_size)])
# print("="*50)
#column constraint
for y in range(sudoku_size):
    for num in range(sudoku_size):
        target_node = ["{}_{}_{}".format(y, x, num) for x in range(sudoku_size)]
        # print(target_node)
        factor = DiscreteFactor(target_node, number_of_random_variable_state, factor_prob)
        graph.add_factors(factor)
        graph.add_edges_from([(target_node[i], factor) for i in range(sudoku_size)])
# print("="*50)
#row constraint
for x in range(sudoku_size):
    for num in range(sudoku_size):
        target_node = ["{}_{}_{}".format(y, x, num) for y in range(sudoku_size)]
        # print(target_node)
        factor = DiscreteFactor(target_node, number_of_random_variable_state, factor_prob)
        graph.add_factors(factor)
        graph.add_edges_from([(target_node[i], factor) for i in range(sudoku_size)])
# print("="*50)
#block constraint
for block_y in range(root_sudoku_size):
    for block_x in range(root_sudoku_size):
        for num in range(sudoku_size):
            target_node = ["{}_{}_{}".format(y, x, num) for y in range(block_y * root_sudoku_size, (block_y + 1) * root_sudoku_size) for x in range(block_x * root_sudoku_size, (block_x + 1) * root_sudoku_size)]
            # print(target_node)
            factor = DiscreteFactor(target_node, number_of_random_variable_state, factor_prob)
            graph.add_factors(factor)
            graph.add_edges_from([(target_node[i], factor) for i in range(sudoku_size)])
graph.check_model()


True

In [ ]:
for y in range(sudoku_size):
    for x in range(sudoku_size):
        if sudoku.init[y][x] == 0:
            for num in range(sudoku_size):
                target_node = "{}_{}_{}".format(y, x, num)
                factor = DiscreteFactor([target_node], [2], [0.5, 0.5])
                graph.add_factors(factor)
                graph.add_edges_from([(target_node, factor)])
        else:
            for num in range(1, sudoku_size + 1):
                target_node = "{}_{}_{}".format(y, x, num)
                if sudoku.init[y][x] == num:
                    factor = DiscreteFactor([target_node], [2], [0.0, 1.0])
                else:
                    factor = DiscreteFactor([target_node], [2], [1.0, 0.0])
                graph.add_factors(factor)
                graph.add_edges_from([(target_node, factor)])

In [7]:
mm = graph.to_markov_model()

In [13]:
from pgmpy.readwrite import UAIWriter
writer = UAIWriter(mm)
writer.write_uai('models/9_9_sudoku_markov.xml')

In [16]:
bm = mm.to_bayesian_model()

/home/takeshi/.pyenv/versions/3.9.4/envs/snn_bayes/lib/python3.9/site-packages/pgmpy/models/MarkovNetwork.py:448: RuntimeWarning: invalid value encountered in long_scalars
  S_by_C = {key: S[key] / C[key] for key in S}


ValueError: maximum supported dimension for an ndarray is 32, found 48

In [ ]:
from pgmpy.inference import ApproxInference
sampling = ApproxInference(bm)
sampling.query()